In [1]:
import numpy as np
import pymongo as pm
import pandas as pd
import os

/Users/judithfan/Library/Python/2.7/lib/python/site-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/Users/judithfan/Library/Python/2.7/lib/python/site-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/Users/judithfan/Library/Python/2.7/lib/python/site-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/Users/judithfan/Library/Python/2.7/lib/python/site-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/Users/judithfan/Library/Python/2.7/lib

In [4]:
import os
import urllib, cStringIO

import pymongo as pm

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
from __future__ import division
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64
import json     

### connect to db

In [5]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['3dObjects']
coll = db['chairs_chatbox']

### basic details about dataset


In [6]:
###### how many total after we started the chairs1k production sample
wave1 = coll.find({'$and': [{'iterationName':'close_only'},{'intendedName':{"$exists": True}},{'eventType':'clickedObj'},{'time':{'$gt':1507674252874}}]}).sort('intendedName')
wave2 = coll.find({'$and': [{'iterationName':'chairs1k'},{'intendedName':{"$exists": True}},{'eventType':'clickedObj'},{'time':{'$gt':1507674252874}}]}).sort('intendedName')
print str(wave1.count() + wave2.count()) + ' chair trials in dataset.'
print str(wave1.count()) + ' wave1 chair trials in dataset.'
print str(wave2.count()) + ' wave2 chair trials in dataset.'

25214 chair trials in dataset.
576 wave1 chair trials in dataset.
24638 wave2 chair trials in dataset.


##### how many unique workers were there?

In [7]:
workers = coll.find({'iterationName': 'close_only'}).distinct('workerId') + \
          coll.find({'iterationName': 'chairs1k'}).distinct('workerId') + \
          coll.find({'iterationName': 'chairs2k_expansion_only'}).distinct('workerId') + \
          coll.find({'iterationName': 'chairs1k_expansion_only'}).distinct('workerId')         
print 'In total, there were {} unique workers.'.format(len(np.unique(workers)))

In total, there were 2124 unique workers.


In [8]:
## how many games did each worker play?
from collections import Counter
worker_games = Counter(workers)
num_per_worker = worker_games.values()

### loop through all games and build group data csv

In [ ]:
## TODO: encoding for worker ID so that we can always map from that anonymized ID to the actual AMT worker ID

In [ ]:
## get for iterationName = 'chairs1k'

########## get list of games
a = coll.find({'$and': [{'iterationName':'chairs1k'},
                    {'eventType':'chatMessage'},
                    {'time':{'$gt':1507674252874}},
                    {'intendedName':{"$exists": True}}
                   ]}).sort('time')
games = a.distinct('gameid')

chairA = []
chairB = []
chairC = []
intendedName = []
clickedObj = []
chatMessage = []
message_counter = []
gameid = []
click_time = []
trialNum = []
condition = []
correct = []
chat_time = []
chat_RT = []
role = []
batch = [] ## ['original' vs. 'patch']

for i,game in enumerate(games):    
    print str(i) + ' ' + game
    clicks = coll.find({'$and': [{'iterationName':'chairs1k'},
                        {'eventType':'clickedObj'},
                        {'time':{'$gt':1507674252874}},
                        {'gameid': game},
                        {'intendedName':{"$exists": True}}                                 
                       ]}).sort('trialNum')  
    
    for click in clicks:
        
        currTrial = click['trialNum']        
        
        chats = coll.find({'$and': [{'iterationName':'chairs1k'},
                            {'eventType':'chatMessage'},
                            {'time':{'$gt':1507674252874}},
                            {'gameid': game},
                            {'trialNum': currTrial},
                            {'intendedName':{"$exists": True}}                                    
                           ]}).sort('time')        
        
        for chat in chats:
            chat_time.append(chat['time'])
            chat_RT.append(chat['reactionTime'])
            chatMessage.append(chat['text'])
            role.append(chat['role'])
            intendedName.append(chat['intendedName'])            
            clickedObj.append(click['clickedName'])
            trialNum.append(click['trialNum'])        
            chairA.append(click['object1Name'])
            chairB.append(click['object2Name'])
            chairC.append(click['object3Name'])  
            click_time.append(click['time'])
            gameid.append(click['gameid'])
            condition.append(click['condition'])
            correct.append(click['correct'])
            if click['time'] < 1522802963803:
                batch.append('original')
            else:
                batch.append('patched')

In [ ]:
########## get list of close-only-games
a = coll.find({'$and': [{'iterationName':'close_only'},
                    {'eventType':'chatMessage'},
                    {'time':{'$gt':1507674252874}}
                   ]}).sort('time')
games = a.distinct('gameid')

for i,game in enumerate(games):    
    print str(i) + ' ' + game
    clicks = coll.find({'$and': [{'iterationName':'close_only'},
                        {'eventType':'clickedObj'},
                        {'time':{'$gt':1507674252874}},
                        {'gameid': game}
                       ]}).sort('trialNum')  
    
    for click in clicks:
        
        currTrial = click['trialNum']        
        
        chats = coll.find({'$and': [{'iterationName':'close_only'},
                            {'eventType':'chatMessage'},
                            {'time':{'$gt':1507674252874}},
                            {'gameid': game},
                            {'trialNum': currTrial}
                           ]}).sort('time')        
        
        for chat in chats:
            chat_time.append(chat['time'])
            chat_RT.append(chat['reactionTime'])
            chatMessage.append(chat['text'])
            role.append(chat['role'])
            intendedName.append(chat['intendedName'])
            
            clickedObj.append(click['clickedName'])
            trialNum.append(click['trialNum'])        
            chairA.append(click['object1Name'])
            chairB.append(click['object2Name'])
            chairC.append(click['object3Name'])  
            click_time.append(click['time'])
            gameid.append(click['gameid'])
            condition.append(click['condition'])
            correct.append(click['correct']) 
            batch.append('original')

#### save out patched chairs1k dataset

In [ ]:
import pandas as pd
X = pd.DataFrame([chairA,
                chairB,
                chairC,
                intendedName,
                clickedObj,
                chatMessage,
                gameid,
                click_time,
                trialNum,
                condition,
                correct,
                chat_time,
                chat_RT,
                role,
                batch])

X = X.transpose()
X.columns = ['chair_a','chair_b','chair_c','target_chair','selected_chair','text','gameid',
            'click_time','trial_num','context_condition','correct','chat_time','chat_RT', 'communication_role','batch']
X.to_csv('chairs1k_group_data_patched.csv',encoding='utf-8')

In [ ]:
### if you need to manually remap batch tags
reallyRun = 0
if reallyRun:
    R = pd.read_csv('chairs1k_group_data_patched.csv')
    d = {'new':'patched',
         'original':'original'}
    R['batch'] = R.batch.map(d)

### add chairs1k doubling dataset

In [ ]:
###### how many total from chairs1k expansion sample
wave3 = coll.find({'$and': [{'iterationName':'chairs1k_expansion_only'},{'intendedName':{"$exists": True}},{'eventType':'clickedObj'},{'time':{'$gt':1507674252874}}]}).sort('intendedName')
print str(wave3.count()) + ' wave3 chair trials in dataset.'

In [ ]:
## get for iterationName = 'chairs1k'

########## get list of games
a = coll.find({'$and': [{'iterationName':'chairs1k_expansion_only'},
                    {'eventType':'chatMessage'},
                    {'time':{'$gt':1507674252874}},
                    {'intendedName':{"$exists": True}}
                   ]}).sort('time')
games = a.distinct('gameid')

chairA = []
chairB = []
chairC = []
intendedName = []
clickedObj = []
chatMessage = []
message_counter = []
gameid = []
click_time = []
trialNum = []
condition = []
correct = []
chat_time = []
chat_RT = []
role = []
batch = [] ## ['original' vs. 'patch' vs. 'doubled']

for i,game in enumerate(games):    
    print str(i) + ' ' + game
    clicks = coll.find({'$and': [{'iterationName':'chairs1k_expansion_only'},
                        {'eventType':'clickedObj'},
                        {'time':{'$gt':1507674252874}},
                        {'gameid': game},
                        {'intendedName':{"$exists": True}}                                 
                       ]}).sort('trialNum')  
    
    for click in clicks:
        
        currTrial = click['trialNum']        
        
        chats = coll.find({'$and': [{'iterationName':'chairs1k_expansion_only'},
                            {'eventType':'chatMessage'},
                            {'time':{'$gt':1507674252874}},
                            {'gameid': game},
                            {'trialNum': currTrial},
                            {'intendedName':{"$exists": True}}                                    
                           ]}).sort('time')        
        
        for chat in chats:
            chat_time.append(chat['time'])
            chat_RT.append(chat['reactionTime'])
            chatMessage.append(chat['text'])
            role.append(chat['role'])
            intendedName.append(chat['intendedName'])            
            clickedObj.append(click['clickedName'])
            trialNum.append(click['trialNum'])        
            chairA.append(click['object1Name'])
            chairB.append(click['object2Name'])
            chairC.append(click['object3Name'])  
            click_time.append(click['time'])
            gameid.append(click['gameid'])
            condition.append(click['condition'])
            correct.append(click['correct'])
            batch.append('doubled')

In [ ]:
import pandas as pd
Y = pd.DataFrame([chairA,
                chairB,
                chairC,
                intendedName,
                clickedObj,
                chatMessage,
                gameid,
                click_time,
                trialNum,
                condition,
                correct,
                chat_time,
                chat_RT,
                role,
                batch])

Y = Y.transpose()
Y.columns = ['chair_a','chair_b','chair_c','target_chair','selected_chair','text','gameid',
            'click_time','trial_num','context_condition','correct','chat_time','chat_RT', 'communication_role','batch']
Y.to_csv('chairs1k_expansion_only_group_data.csv',encoding='utf-8',index=False)

### combine accumulated chairs1k dataset (collected April 17 - April 20 2018)

In [ ]:
## load in patched data from first two waves
Z = pd.read_csv('chairs1k_group_data_patched.csv')

## load in expansion only    
Y = pd.read_csv('chairs1k_expansion_only_group_data.csv')   

## save out all chairs1k data
W = pd.concat([Z,Y])
W.to_csv('chairs1k_group_data.csv',index=False)

### preprocess chairs2k dataset (collected April 23 - April 24 2018)

In [ ]:
###### how many total from chairs1k expansion sample
wave4 = coll.find({'$and': [{'iterationName':'chairs2k_expansion_only'},{'intendedName':{"$exists": True}},{'eventType':'clickedObj'},{'time':{'$gt':1507674252874}}]}).sort('intendedName')
print str(wave4.count()) + ' wave4 chair trials in dataset.'

In [ ]:
## get for iterationName = 'chairs2k'

########## get list of games
a = coll.find({'$and': [{'iterationName':'chairs2k_expansion_only'},
                    {'eventType':'chatMessage'},
                    {'time':{'$gt':1507674252874}},
                    {'intendedName':{"$exists": True}}
                   ]}).sort('time')
games = a.distinct('gameid')

chairA = []
chairB = []
chairC = []
intendedName = []
clickedObj = []
chatMessage = []
message_counter = []
gameid = []
click_time = []
trialNum = []
condition = []
correct = []
chat_time = []
chat_RT = []
role = []
batch = [] ## ['original' vs. 'patch']

for i,game in enumerate(games):    
    print str(i) + ' ' + game
    clicks = coll.find({'$and': [{'iterationName':'chairs2k_expansion_only'},
                        {'eventType':'clickedObj'},
                        {'time':{'$gt':1507674252874}},
                        {'gameid': game},
                        {'intendedName':{"$exists": True}}                                 
                       ]}).sort('trialNum')  
    
    for click in clicks:
        
        currTrial = click['trialNum']        
        
        chats = coll.find({'$and': [{'iterationName':'chairs2k_expansion_only'},
                            {'eventType':'chatMessage'},
                            {'time':{'$gt':1507674252874}},
                            {'gameid': game},
                            {'trialNum': currTrial},
                            {'intendedName':{"$exists": True}}                                    
                           ]}).sort('time')        
        
        for chat in chats:
            chat_time.append(chat['time'])
            chat_RT.append(chat['reactionTime'])
            chatMessage.append(chat['text'])
            role.append(chat['role'])
            intendedName.append(chat['intendedName'])            
            clickedObj.append(click['clickedName'])
            trialNum.append(click['trialNum'])        
            chairA.append(click['object1Name'])
            chairB.append(click['object2Name'])
            chairC.append(click['object3Name'])  
            click_time.append(click['time'])
            gameid.append(click['gameid'])
            condition.append(click['condition'])
            correct.append(click['correct'])
            batch.append('2k')

In [ ]:
import pandas as pd
Y = pd.DataFrame([chairA,
                chairB,
                chairC,
                intendedName,
                clickedObj,
                chatMessage,
                gameid,
                click_time,
                trialNum,
                condition,
                correct,
                chat_time,
                chat_RT,
                role,
                batch])

Y = Y.transpose()
Y.columns = ['chair_a','chair_b','chair_c','target_chair','selected_chair','text','gameid',
            'click_time','trial_num','context_condition','correct','chat_time','chat_RT', 'communication_role','batch']


In [ ]:
## check to make sure this dataframe has indices

In [ ]:
Y.to_csv('chairs2k_expansion_only_group_data.csv',encoding='utf-8',index=False)

In [ ]:
from collections import Counter
b = zip(Y['chair_a'],Y['chair_b'],Y['chair_c'],Y['context_condition'])

In [ ]:
triplet_id = []
for i,d in Y.iterrows():
    triplet_id.append(d['chair_a']+d['chair_b']+d['chair_c']+d['context_condition']+d['target_chair'])

In [ ]:
print len(triplet_id)
print len(np.unique(triplet_id))
c = Counter(triplet_id).values()
c = np.array(c)
plt.hist(c,bins=13)
r = plt.xticks(np.arange(0,12))

### Combine chairs1k and chairs2k data to create accumulated dataset

In [ ]:
## load in patched data from first two waves
Z = pd.read_csv('chairs1k_group_data_doubled.csv')
# if Z['Unnamed: 0'].any():
#     Z = Z.drop(['Unnamed: 0'], axis = 1)

## load in expansion only    
Y = pd.read_csv('chairs2k_expansion_only_group_data.csv')   

## save out all chairs1k data
W = pd.concat([Z,Y])
W.to_csv('chairs2k_group_data.csv',index=False)

In [ ]:
## load in patched data from first two waves
Z = pdc.read_csv('chairs1k_group_data_patched.csv')

## load in expansion only    
Y = pd.read_csv('chairs2k_expansion_only_group_data.csv')   

## save out all chairs1k data
W = pd.concat([Z,Y])
W.to_csv('chairs2k_group_data_rounded.csv',index=False)

### Sanity checks on data organization

In [ ]:
## the three chairs ID's should be unique
from collections import defaultdict
all_triplets = defaultdict(list)
for i,d in Y.iterrows():

    tokens_1 = d['chair_a'].split('_')
    tokens_2 = d['chair_b'].split('_')
    tokens_3 = d['chair_c'].split('_')
    
    triplets = []        
    triplets.append(tokens_1[-1])    
    triplets.append(tokens_2[-1])
    triplets.append(tokens_3[-1])
    assert(len(np.unique(triplets)) == 3)
    
    assert(d['context_condition'] == tokens_1[0] == tokens_2[0] == tokens_3[0])
    
    all_triplets[(d['context_condition'], d['target_chair'])].append(np.array(triplets, dtype=object))
    
#     assert d['clickedObj'] in  [d['chairsA'], d['chairsB'], d['chairsC']]

#     assert(d['target_chair'] in  [d['chairsA'], d['chairsB'], d['chairsC']])
    
#     if d['intendedName'] == d['clickedObj']:
#         assert(d['correct'])
#     else:
#         assert(not d['correct'])
    
    ## check the total number of chair ID's

    ## check the total number of unique triplets (where target identity matters AND where it does not matter)

    


In [ ]:
c_close = 0 
c_far = 0
close_triplets = []
far_triplets = []
for l in all_triplets.keys():
    
#     a  = all_triplets[l][0].split('_')[-1]
#     b = all_triplets[l][1].split('_')[-1]
#     c = all_triplets[l][2].split('_')[-1]
    
    if l[0] == 'close':
        c_close +=1
        close_triplets.append(np.sort(all_triplets[l]))
    elif l[0] == 'far':
        c_far += 1
        far_triplets.append(np.sort(all_triplets[l]))
    else:
        assert(False)
        

In [ ]:
far_triplets = np.vstack(close_triplets)

In [ ]:
len(np.unique(far_triplets))

### prep for wave3: load in original data from waves 1 and 2, check which triplets we need to "patch" by collecting more data

In [ ]:
X = pd.read_csv('chairs1k_group_data.csv')
X = X.drop(['Unnamed: 0'],axis=1)

In [ ]:
X.columns

In [ ]:
## compose chairs into triplets and count how many utterances we have for each triplet
abct = []
for i,d in X.iterrows():
    if i%6000==0:
        print '{} of {} chairs'.format(i,len(X))
    a = d['chair_a']
    b = d['chair_b']
    c = d['chair_c']
    t = d['target_chair']
    abct.append(tuple((a,b,c,t)))  

In [ ]:
## how many unique triplets do we have utterances on?
print '{} unique triplets'.format(len(list(set(abct))))

In [ ]:
## add triplet tuple to dataframe
triplet_list = triplets
X['triplet_plus_target'] = abct

In [ ]:
## convert to flattened dataframe with list of gameids that saw this triplet
X2 = pd.DataFrame(X.groupby('triplet_plus_target')['gameid'].unique())
X2.reset_index(inplace=True)

In [ ]:
## break out triplet_plus_target into separate triplet and target columns
triplet = []
target = []
for i,d in X2.iterrows():
    triplet.append(list(d['triplet_plus_target'][:3]))
    target.append(d['triplet_plus_target'][-1])

    ## add triplet and target columns to dataframe    
X2['triplet'] = triplet
X2['target'] = target    

In [ ]:
## load in the chairs1k json
import json
J = json.loads(open('../stimuli/chairs1k.js',mode='ru').read())

In [ ]:
## loop through entries in J and rewrite the games field to actually contain 
## the list of gameids that have seen this triplet
for j in J:
    targ_index = j['target_status'].index('target')
    chairs = [i.split('.')[0] for i in j['filename']]
    targ = chairs[targ_index]
    ## find the index in X2 that corresponds to this triplet,target tuple and get list of gameids
    for i,x in enumerate(zip(X2['triplet'].values,X2['target'].values)):
        if (x[0]==chairs) & (x[1]==targ):
            ## retriee list of games that have seen this triplet
            gameid_list = X2.iloc[i]['gameid']
            j['games'] = list(gameid_list)

In [ ]:
## save out J as annotated version of dataset
import json
with open('../stimuli/chairs1k_annotated_032718.js', 'w') as fout:
    json.dump(J, fout)

#### check on weird game with missing text data Aug 9 2018

In [25]:
game = '6728-eac16fc7-3ed1-4a68-8d6b-f86af67bd620'
clicks = coll.find({'$and': [{'iterationName':'chairs2k_expansion_only'},
                    {'intendedName':{"$exists": True}},
                    {'eventType':'clickedObj'},
                    {'time':{'$gt':1507674252874}},
                    {'gameid': game}]}).sort('intendedName')

chatMessage = []
correct = []

for click in clicks:
    currTrial = click['trialNum']        
    chats = coll.find({'$and': [{'iterationName':'chairs2k_expansion_only'},
                        {'eventType':'chatMessage'},
                        {'time':{'$gt':1507674252874}},
                        {'gameid': game},
                        {'trialNum': currTrial},
                        {'intendedName':{"$exists": True}}                                    
                       ]}).sort('time') 
    
    for chat in chats:
#         chat_time.append(chat['time'])
#         chat_RT.append(chat['reactionTime'])
        chatMessage.append(chat['text'])
#         role.append(chat['role'])
#         intendedName.append(chat['intendedName'])            
#         clickedObj.append(click['clickedName'])
#         trialNum.append(click['trialNum'])        
#         chairA.append(click['object1Name'])
#         chairB.append(click['object2Name'])
#         chairC.append(click['object3Name'])  
#         click_time.append(click['time'])
#         gameid.append(click['gameid'])
#         condition.append(click['condition'])
        correct.append(click['correct'])
#         batch.append('2k')    

In [27]:
zip(chatMessage, correct)

[(u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', False),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u'  ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u'chair with thin bar arm rest', True),
 (u'the couch with the arms that have slots in them', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', True),
 (u' ', Tr

In [32]:
len(chatMessage[-1])

1